In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
import time
import random
import requests
import os
import glob
import re
import csv


class EBSCO_CRAWLER():

    def __init__(self, driver_path, SaveFile_Path):
        self.url_to_crawl = 'http://search.ebscohost.com/login.asp?profile=ehost&defaultdb=lxh'
        self.driver_path = driver_path
        self.SaveFile_Path = SaveFile_Path
        self.keys = keys
        self.gg = gg

    def start(self):
        self.browser = webdriver.Chrome(self.driver_path)
        self.browser.maximize_window()

    def get_into_page(self, url):
        self.browser.get(url)
        time.sleep(5)
        #先點一次查詢，進到搜尋主頁面
        self.browser.find_element(By.ID, 'selectDBLink').click()        
        time.sleep(5)
        self.browser.find_element(By.ID, 'selectAll').click()        
        self.browser.find_element(By.ID, 'btnOKTop').click()        
        time.sleep(5)

    def search_work(self):
        self.browser.find_element(By.ID, "Searchbox1").send_keys(keys)
        self.browser.find_element(By.ID, "SearchButton").click()
        soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        page_str = soup.find('h1', {'class': 'page-title alt'})
        page_text = page_str.text
        pattern = '[0-9]+'
        page_re = re.findall(pattern, page_text)[2]
        page_int = int(page_re)#筆數
        page_last = page_int % 50
        if page_int % int(50) == 0:
            self.page_all = page_int // int(50)
        else:
            self.page_all = (page_int // int(50)) + 1  #頁數
        print('共 ' + str(page_int) + ' 筆')
        print('有 ' + str(self.page_all) + ' 頁')
        print('最後一頁剩 ' + str(page_last) + ' 筆')

    def download_file(self, soup):
        result_list = []
        ttl_list = soup.find_all('a', {'class': 'title-link color-p4'})
        content_list = soup.find_all('div', {'class': 'display-info'})
        for row in range(0, len(ttl_list)):
            title = ttl_list[row].text  #標題
            content = content_list[row].text  #作者、期刊名稱、頁數、DOI
            remove = content[content.find('Add to folder'):len(content)]
            content = ' '.join(
                content.replace(remove, "").replace("\n", '').split())
            db = content[content.find('Database: '):len(content)]
            info = content[0:content.find(', Database: ')]
            result_list.append([(self.page - 1) * 50 + row + 1, title,
                                content, info,db])
        return result_list

    def roll_page(self):  #翻頁並進入search_wor和download_file的步驟
        self.page = 1
        i = 1
        res_process = []
        res_all = pd.DataFrame()
        while self.page <= self.page_all:
            soup = BeautifulSoup(self.browser.page_source, 'html.parser')
            result_list = self.download_file(soup)
            time.sleep(5)
            result_df = pd.DataFrame(result_list)
            res_process = pd.DataFrame(res_process)
            res_process = res_process.append(result_df)
            self.page = self.page + 1
            try:
                self.browser.find_element(By.ID,
                    "ctl00_ctl00_MainContentArea_MainContentArea_bottomMultiPage_lnkNext"
                ).click()
                time.sleep(random.randint(5, 25))  #避開伺服器察覺相同頻率爬取資料  ✔️✔️✔️重要
            except:
                res_all = res_all.append(res_process)
                res_all.columns = ["筆數","原始題目","資訊","資訊2","資料庫"]
                try:
                    res_all.to_csv(SaveFile_Path + '\\' + SaveFile_Name + "_" +
                                   "_" + time.strftime("%m%d") + "_" +
                                   time.strftime("%H%M") + ".csv",
                                   encoding="utf_8_sig",
                                   index=False)  #儲存結果
                    print("中途存檔成功")  
                    break
                except:
                    print("存檔失敗，請檢察結果檔是否正在開啟，或處於無法被寫入的狀態。")
            if self.page == self.page_all:
                print("此步驟為爬取最終頁資料")
                soup = BeautifulSoup(self.browser.page_source, 'lxml')
                result_list = self.download_file(soup)
                time.sleep(5)
                result_df = pd.DataFrame(result_list)
                res_process = pd.DataFrame(res_process)
                res_process = res_process.append(result_df)
                res_all = res_all.append(res_process)
                res_all.columns = ["筆數","原始題目","資訊","資訊2","資料庫"]
                try:
                    res_all.to_excel(SaveFile_Path + '\\' + SaveFile_Name + "_" +
                                   "_" + time.strftime("%m%d") + "_" +
                                   time.strftime("%H%M") + ".xlsx",
                                   encoding="utf_8_sig",
                                   index=False)  #儲存結果
                    print("存檔成功")  #最終檔名會加上 1.月份 2.日期 3.時間(時分)
                    break
                except:
                    print("存檔失敗，請檢察結果檔是否正在開啟，或處於無法被寫入的狀態。")

    def close_driver(self):
        self.browser.quit()
        print('成功爬取EBSCO - ' + Dataset)

    def step(self):
        self.start()
        print('正在爬取EBSCO')
        self.get_into_page(self.url_to_crawl)
        self.search_work()
        self.roll_page()
        self.close_driver()


if __name__ == '__main__':
    driver_path = r'C:\Users\user\Desktop\py\Driver\chromedriver.exe'  #chromedriver位置
    SaveFile_Path = r'C:\Users\user\Desktop\py\衍伸書目\EBSCO'  #要儲存檔案的路徑
    keys_path = r'C:\Users\user\Desktop\py\指令整理.xlsx'  #指令
    gg = 1
    ALLkeys = pd.read_excel(keys_path)
    ALLkeys = ALLkeys[120:144]
    keys = ALLkeys['Final'].iloc[gg - 1]
    Dataset = ALLkeys['Dataset'].iloc[gg - 1]
    SaveFile_Name = r'EBSCO' + Dataset
    go = EBSCO_CRAWLER(driver_path, SaveFile_Path)
    go.step()

# 1. 台灣社會變遷基本調查(TSCS)
# 2. 華人家庭動態資料庫(PSFD)
# 3. 台灣教育長期追蹤(TEPS)
# 4. 特殊教育長期追蹤資料庫(SNELS)
# 5. 家庭收支調查(SFIE)
# 6. 台灣世界價值觀調查(TWVS)
# 7. 台灣青少年成長歷程研究(TYP)
# 8. 台灣選舉調查資料(TEDS)
# 9. 「台灣教育長期追蹤資料庫」後續調查(TEPS-B)
# 10. 人力資源調查(MPS)
# 11. 人力運用調查(MUS)
# 12. 婦女婚育與就業調查(WMFES)
# 13. 個人家戶數位機會(落差)(數位落差)(DDS)
# 14. 台灣貧窮兒少資料庫(TDCYP)
# 15. 台灣傳播調查資料庫(TCS)
# 16. International Social Survey Programme (ISSP)
# 17. 幼兒發展調查資料庫建置計畫
# 18. 國人旅遊狀況調查
# 19. 來臺旅客消費及動向調查
# 20. 台灣社會意向調查
# 21. 台灣營養健康狀況變遷調查
# 22. 我國大學生政治社會化的定群追蹤研究
# 23. 台灣政府文官調查
# 24. 青少年藥物濫用之起因

C:\Temp\TEMP\ipykernel_1260\3822443864.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.browser = webdriver.Chrome(self.driver_path)


正在爬取EBSCO
共 301 筆
有 7 頁
最後一頁剩 1 筆


C:\Temp\TEMP\ipykernel_1260\3822443864.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Temp\TEMP\ipykernel_1260\3822443864.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Temp\TEMP\ipykernel_1260\3822443864.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_all = res_all.append(res_process)


中途存檔成功
成功爬取EBSCO - 台灣社會變遷基本調查(TSCS)
